## Interpteter for both Boolean and numeric Expression

Install the lark-parser with

> pip install lark-parser 

### Grammar

In [22]:
from lark import Lark, InlineTransformer 


""" gexp is boolexp grammar on top of exp grammar: 
    numeric operators have higher precedence than boolean operators.
"""
grammar_gexp = """
    ?start: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolnot
        | booland "and" boolnot -> andop
    ?boolnot: boolcmp
        | "not" boolnot     -> notop
    ?boolcmp: expcmp
        | boolcmp "==" expcmp -> eq
    ?expcmp: exp
        | expcmp "<=" exp      -> lte
        | expcmp "<" exp       -> lt
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul
        | product "/" atom  -> div
    ?atom: NUMBER           -> number
         | "-" atom         -> neg
         | "True"           -> true
         | "False"          -> false
         | NAME             -> var
         | "(" boolexp ")"
    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

### Parser
Implementing the functions described by grammar productions

In [23]:
class Tree_GExp(InlineTransformer):
    number = float

    def var(self, name):
        return ['variable', str(name)]
    
    def false(self):
        return ['bool_const', False]
    
    def true(bool_const):
        return ['bool_const', True]
    
    def orop(self, left, right):
        return ['or', left, right]
    
    def andop(self, left, right):
        return ['and', left, right]

    def notop(self, value):
        return ['not', value]

    def lte(self, left, right):
        return ['<=', left, right]

    def lt(self, left, right):
        return ['<', left, right]

    def eq(self, left, right):
        return ['==', left, right]

    def number(self, value):
        return ['number_const', float(value)]

    def add(self, left, right):
        return ['+', left, right]
    
    def sub(self, left, right):
        return ['-', left, right]
    
    def mul(self, left, right):
        return ['*', left, right]
    
    def div(self, left, right):
        return ['/', left, right]
    
    def neg(self, left):
        return ['-', left]

In [24]:
def get_parser_gexp():
    parser = Lark(grammar_gexp, parser='lalr', transformer=Tree_GExp())
    return parser.parse

In [25]:
parse = get_parser_gexp()
    
E1 = parse("3 <= 5")
E1

['<=', ['number_const', 3.0], ['number_const', 5.0]]

In [26]:
E2 = parse("x < 2 or 3 <= y+2")
E2

['or',
 ['<', ['variable', 'x'], ['number_const', 2.0]],
 ['<=',
  ['number_const', 3.0],
  ['+', ['variable', 'y'], ['number_const', 2.0]]]]

In [27]:
E3 = parse("False == True")
E3

['==', ['bool_const', False], ['bool_const', True]]

In [28]:
E4 = parse("3<4 == 2<3") # C semantics (not Python!)
E4

['==',
 ['<', ['number_const', 3.0], ['number_const', 4.0]],
 ['<', ['number_const', 2.0], ['number_const', 3.0]]]

In [29]:
E5 = parse("5 or 4") # C semantics (not Python!)
E5

['or', ['number_const', 5.0], ['number_const', 4.0]]

### Interpreter

- Objective: define solve_exp(E, sigma)

How to get to a function that given an AST and a sigma evaluate the Expression in sigma?

In [30]:
def solve_exp(exp, sigma):
    op = exp[0]
    """ Algebra of numbers """
    if op=='number_const':
        return exp[1]
    if op=='variable':
        return sigma[exp[1]]
    if op=='+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==3:
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==2:
        return -solve_exp(exp[1], sigma)
    if op=='*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op=='/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    """ Algebra of Booleans """
    if op=='bool_const':
        return exp[1]
    if op=='or':
        return bool(solve_exp(exp[1], sigma)) or bool(solve_exp(exp[2], sigma))
    if op=='and':
        return bool(solve_exp(exp[1], sigma)) and bool(solve_exp(exp[2], sigma))
    if op=='not':
        return not solve_exp(exp[1], sigma)
    if op=='==':
        return solve_exp(exp[1], sigma) == solve_exp(exp[2], sigma)
    if op=='<=':
        return solve_exp(exp[1], sigma) <= solve_exp(exp[2], sigma)
    if op=='<':
        return solve_exp(exp[1], sigma) < solve_exp(exp[2], sigma)

In [31]:
sigma = {} # empty interpretation
print('E1 = ', solve_exp(E1, sigma))

E1 =  True


In [32]:
sigma = { 'x':2, 'y':1 }
print('E2 = ', solve_exp(E2, sigma))

E2 =  True


In [33]:
sigma = {} # empty interpretation
print('E3 = ', solve_exp(E3, sigma))

E3 =  False


In [34]:
print('E4 = ', solve_exp(E4, sigma))

E4 =  True


In [35]:
print('E5 = ', solve_exp(E5, sigma))

E5 =  True


### Type Checking (C like): static semantics
- Objective: ensure that Boolean and Numeric Expressions are applied on homogeneous types

In [36]:
def type_check(exp, iota):
    op = exp[0]
    
    """ Algebra of numbers """
    if op=='number_const':
        return 'number'
    if op=='variable':
        return iota[exp[1]]
    if op=='+' or op=='-' or op=='*' or op=='/':
        type_left = type_check(exp[1], iota)
        if type_left != 'number':
            raise Exception("Expected: number Actual: " + type_left + " in expr: ", exp[1])
        if len(exp)==3:
            type_right = type_check(exp[2], iota)
            if type_right != 'number':
                raise Exception("Expected: number Actual: " + type_right +" in expr: ", exp[2])
        return 'number'
    
    """ Algebra of Booleans """
    if op=='bool_const':
        return 'bool'
    if op=='not' or op=='and' or op=='or':
        type_left = type_check(exp[1], iota)
        if type_left != 'bool':
            raise Exception("Expected: bool Actual: " + type_left + " in expr: ", exp[1])
        if len(exp)==3:
            type_right = type_check(exp[2], iota)
            if type_right != 'bool':
                raise Exception("Expected: bool Actual: " + type_right +" in expr: ", exp[2])
        return 'bool'
    if op=='==' or op=='<=' or op=='<':
        type_left = type_check(exp[1], iota)
        type_right = type_check(exp[2], iota)
        if type_left != type_right:
            raise Exception("Expected: " + type_left + " Actual: " + type_right +" in expr: ", exp[2])
        return 'bool'

In [37]:
iota = { 'x':'number', 'y':'number' }
print('type_check(E2) = ', type_check(E2, iota))

type_check(E2) =  bool


In [38]:
iota = { }
print('type_check(E5) = ', type_check(E5, iota))

Exception: ('Expected: bool Actual: number in expr: ', ['number_const', 5.0])